# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

### CORRELATING SKILL WITH SALARY 

The prediction of Salary structure in Human Resource department ( HR) of a company  contains two fold objectives: 

    -To  attract and retain the talented employees 
    
The concept of demand and supply of work force  needs  to be examined. The key intakes in HR department include - identifying the responsibilities of each  job, appointing the right man to the right post  and evolving a salary package that suits the best in the industry,  based on parameters like qualifications, industry, job location.  This is vital to the HR department as it helps the company in reducing the unnecessary cost while retaining the best talent. 

On the other hand, Salary prediction can also help the job seekers. It helps job seekers to  take more informed decisions and update their skills/knowledge to maximize salary.

**It is a win – win formula for both employers and job seekers.** 

In [3]:
#import libraries

#Analysis packages
import pandas as pd
import sklearn as sk
import numpy as np

#Visualization packages
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

#Sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor


#your info here
__author__ = "Rahul Vashsith"
__email__ = "rahulvashisth3@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [1]:
#load the data into a Pandas dataframe

### ---- 3 Clean the data ----

In [2]:
#look for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it

### ---- 4 Explore the data (EDA) ----

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [15]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [1]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data